In [1]:
import pandas as pd
import numpy as np
import glob


from tqdm import tqdm

import sys
from tqdm import tqdm

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np

import itertools

sys.path.append('../')

from sys_utils import load_model
from data_model_region import Region
from data_model import Individual
import typing as t
import random
from tqdm import tqdm
from bunka_logger import logger

In [2]:
checkpoint_path = '../checkpoints_dev'


In [3]:
individuals_test = load_model(
        Individual, name=checkpoint_path + "/individuals_corrected.jsonl"
    )

In [110]:
checkpoint_path = '../checkpoints_dev'

regions = load_model(
        Region, name=checkpoint_path+"/regions.jsonl"
    )

individuals = load_model(
        Individual, name=checkpoint_path + "/individuals.jsonl"
    )

In [8]:
ind = [x for x in individuals if x.id.wikidata_id == 'Q7263938']

ind[0].regions

['re_south_italy',
 're_greek_world',
 're_mediterreanean',
 're_western_europe',
 're_southwestern_europe',
 're_italy']

In [500]:
ind[0].id.name

'Python of Catana'

In [9]:
adapted_country = {'Arab Countries':['re_arabic_world', 're_muslim_world'],
 'Arab World':['re_arabic_world', 're_muslim_world'],
 'Armenian':['re_arabic_world', 're_muslim_world'],
 'Danish':['re_nordic_countries', 're_western_europe', 're_northwestern_europe', 're_denmark'],
 'English':['re_united_kingdom', 're_western_europe', 're_northwestern_europe'],
 'French':['re_france', 're_western_europe', 're_southwestern_europe'],
 'Greek ':['re_greek_world', 're_greece'],
 'Greek World':['re_greek_world', 're_greece'],
 'Indian Countries':['re_indian_world'],
 'Italy': ['re_italy', 're_western_europe', 're_southwestern_europe'],
 'Latin World':['re_latin'],
 'Low Countries':['re_low_countries', 're_western_europe', 're_northwestern_europe'],
 'Persian World':['re_persian_world'],
 'Portugal':['re_portugal', 're_western_europe', 're_southwestern_europe'],
 'Russian':['re_eastern_europe', 're_slav_world'],
 'Russsian':['re_eastern_europe', 're_slav_world'],
 'Spain':['re_spain','re_western_europe', 're_southwestern_europe'],
 'Spanish':['re_spain','re_western_europe', 're_southwestern_europe'],
 'Suisse':['re_german_world', 're_western_europe', 're_northwestern_europe', 're_switzerland'],
 'Turkish World':['re_greek_world', 're_ottoman_turkey'],
 'UK':['re_united_kingdom', 're_western_europe', 're_northwestern_europe']}


In [11]:
def manual_cleaning_of_individuals(env_path = '../data_env'):
    
    paths = glob.glob(env_path + '/manual_individuals_check/*')

    list_df = [pd.read_csv(x) for x in paths]
    df_old = pd.concat([x for x in list_df])
    df_old = df_old.reset_index(drop=True)

    df_old = df_old[['individual_id', 'meta_country', 'productive_year']].drop_duplicates()
    df_old['birthyear'] = df_old['productive_year'] - 35
    df_old = df_old.dropna().reset_index(drop=True)

    df_old['region_code_corrected'] = df_old['meta_country'].apply(lambda x : adapted_country.get(x))
    df_old = df_old.rename(columns = {'individual_id':'wikidata_id'})
    df_old = df_old.drop('meta_country', axis=1)

    #df_new = pd.merge(df_old, df_check, on = 'wikidata_id', how = 'outer')
    #df_new = df_new[~df_new['region_code_corrected'].isna()]

    df_final = df_old[['wikidata_id', 'region_code_corrected']].copy()
    df_final = df_final.explode('region_code_corrected')
    df_final = df_final.drop_duplicates().reset_index(drop=True)
    
    return df_final

In [12]:
df_wiki_new_region = manual_cleaning_of_individuals()

In [15]:
def modify_existing_data(df_wiki_new_region, df_wiki):
    
    # Merge the dataframes on the 'wikidata_id' column
    merged_df = pd.merge(df_wiki, df_wiki_new_region, on='wikidata_id', how='left')
    merged_df = merged_df.drop_duplicates()

    # Update the 'region_code' column based on the values in 'region_code_corrected'
    merged_df.loc[merged_df['region_code_corrected'].notnull(), 'region_code'] = merged_df['region_code_corrected']
    merged_df = merged_df.drop('region_code_corrected', axis=1)
    merged_df = merged_df.drop_duplicates()
    
    return merged_df

In [164]:
def pipeline_manual_cleaning_region_global(individuals:t.List[Individual], env_path: str)->t.List[Individual]:
    

    # Get Exisiting Data
    df_wiki = [
        {
            "wikidata_id": x.id.wikidata_id,
            "region_code": x.regions
        }
        for x in individuals
    ]

    df_wiki = pd.DataFrame(df_wiki)
    df_wiki = df_wiki.explode('region_code')
    
    # Get new data
    df_wiki_new_region = manual_cleaning_of_individuals(env_path)
    
    # Merge exisitng and new Data and modify the region_name
    merged_df = modify_existing_data(df_wiki_new_region, df_wiki)
    merged_df = merged_df.groupby(['wikidata_id'])['region_code'].apply(list).reset_index()
    

    # Add manual cleaning 
    manual_cleaning = pd.read_csv(env_path + '/manual_individuals_check/ENS - Cultural Index - Countries Databases - individuals_cleaned.csv')

    manual_cleaning = manual_cleaning[['wikidata_id', 'region_code_corrected']]
    manual_cleaning['region_code_corrected'] = manual_cleaning['region_code_corrected'].apply(lambda x : x.split(','))

    merged_df = pd.merge(merged_df, manual_cleaning, on = 'wikidata_id', how = 'outer')
    merged_df.loc[merged_df['region_code_corrected'].notnull(), 'region_code'] = merged_df['region_code_corrected']
    merged_df = merged_df.drop('region_code_corrected', axis=1)
    
    
    # Fix the format of [None]
    df_all_ids = merged_df[['wikidata_id']].copy()
    df_change_none = merged_df.explode('region_code')
    df_change_none['region_code'][df_change_none['region_code']=='None'] = np.nan
    df_change_none = df_change_none.dropna()
    df_change_none = df_change_none.groupby('wikidata_id')['region_code'].apply(list).reset_index()

    merged_df = pd.merge(df_all_ids, df_change_none, on = 'wikidata_id', how = 'outer')



    # create a format to insert into the Indiviudal Model
    id_regions = {x['wikidata_id']:x['region_code'] for x in merged_df.to_dict(orient='records')}

    logger.info("Replacing region_code for corrected individuals")
    
    for ind in tqdm(individuals):
        region = id_regions.get(ind.id.wikidata_id, None)
        
        if isinstance(region, list):
            ind.regions = region     
        else:
            ind.regions = None
        
    return individuals

In [165]:
final_individuals = pipeline_manual_cleaning_region_global(individuals, env_path = '../data_env')

INFO      |2023-04-07 17:09:37|{}|pipeline_manual_cleaning_region_global|Replacing region_code for corrected individuals
100%|████████████████████████████████| 225636/225636 [00:02<00:00, 103528.54it/s]


In [166]:
ind = [x for x in final_individuals if x.id.wikidata_id == 'Q9441']

In [173]:
from sys_utils import save_model

In [174]:
save_model(final_individuals, name="../checkpoints_dev/individuals_corrected.jsonl")

In [175]:
individuals_test = load_model(
        Individual, name=checkpoint_path + "/individuals_corrected.jsonl"
    )

In [185]:
ind = [x for x in final_individuals if x.id.wikidata_id == 'Q8007920']